Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [529]:
from random import random, choice, choices, randint
from dataclasses import dataclass
from copy import copy, deepcopy
from math import ceil, floor

import lab9_lib

In [530]:
TOURNAMENT_SIZE = 2
GENOME_LENGTH = 1000

In [531]:
@dataclass
class Individual:
    genotype: list[int]
    fitness: float
    age: float

    def __hash__(self) -> int:
        return hash(''.join(map(str, self.genotype))
)

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate_multi(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    for _ in range(20):
        pos = randint(0, GENOME_LENGTH-1)
        offspring.genotype[pos] = 1 - offspring.genotype[pos]
    return offspring

def mutate_two_cut(ind: Individual) -> Individual:
    cut_point_A = randint(0, GENOME_LENGTH-1)
    cut_point_B = randint(cut_point_A, GENOME_LENGTH-1)
    offspring = deepcopy(ind)
    for pos in range(cut_point_B-cut_point_A):
        if(random() < 0.7):
            offspring.genotype[pos] = 1 - offspring.genotype[pos]
    assert len(offspring.genotype) == GENOME_LENGTH
    return offspring

def two_cut_uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    new_genome = list()
    cut_point_A = randint(0, GENOME_LENGTH-1)
    cut_point_B = randint(cut_point_A, GENOME_LENGTH-1)
    for i in range(cut_point_B-cut_point_A):
        gene = choice([ind1.genotype[i], ind2.genotype[i]])
        new_genome.append(gene)
    offspring = Individual(genotype=ind1.genotype[:cut_point_A] + new_genome + ind1.genotype[cut_point_B:], fitness=0, age=1)
    assert len(offspring.genotype) == GENOME_LENGTH
    return offspring

#Cheating
def duplicate_mutation(ind: Individual) -> Individual:
    cut_point_A = randint(0, GENOME_LENGTH-2)
    cut_point_B = randint(cut_point_A+1, GENOME_LENGTH-1)
    times = ceil(GENOME_LENGTH/(cut_point_B-cut_point_A))
    offspring = Individual(genotype=((ind.genotype[cut_point_A:cut_point_B])*times)[:GENOME_LENGTH], fitness=0)
    assert len(offspring.genotype) == GENOME_LENGTH
    return offspring

In [532]:
fitness = lab9_lib.make_problem(2)

In [533]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
NICHE_STEP = 20
MUTATION_PROBABILITY = .2
AGING_RATE = 0.001

In [534]:
pools = [list() for _ in range(0,101,NICHE_STEP)] #0,101,NICHE STEP for fitness == 1

In [535]:
for generation in range(800):
    if(generation%50==0):
        print(f"Gen {generation+1}")
    offspring = [Individual(genotype=[choice((0, 1)) for _ in range(GENOME_LENGTH)],fitness=0,age=1) for _ in range(OFFSPRING_SIZE)]
    offsprings = [list() for _ in range(0,101,NICHE_STEP)]
    for o in offspring:
        o.fitness = fitness(o.genotype)
        pools[floor(o.fitness*100)//NICHE_STEP].append(o)
    for pool in pools:
        for ind in pool:
            ind.age -= AGING_RATE
            if random() < MUTATION_PROBABILITY:
                o = choice([mutate_multi,mutate_two_cut])(ind)
                o.fitness = fitness(o.genotype)
            else:
                p1 = select_parent(choices([pools[i] for i, pool in enumerate(pools) if len(pool)],[i+1 for i, pool in enumerate(pools) if len(pool)], k=1)[0]) #Parent can be choosen among every pool but high-scored are more probable
                o = two_cut_uniform_xover(p1, ind)
                o.fitness = fitness(o.genotype)
            offsprings[floor(o.fitness*100)//NICHE_STEP].append(o)
    for idx, pool in enumerate(pools):
        pool.extend(offsprings[idx])
        pool.sort(key=lambda i: i.fitness*i.age, reverse=True)
        pool = list(dict.fromkeys(pool)) #Remove duplicates
        pools[idx] = pool[:POPULATION_SIZE]
    
    if(generation%50==0):
        for idx, pool in enumerate(pools):
            print(f"Pool {idx} top {5 if len(pool) >= 5 else len(pool)}")
            for i in range(5 if len(pool) >= 5 else len(pool)):
                print(f"ind {i})\t fitness:{pool[i].fitness} genotype:{pool[i].genotype}")
        print(f"{fitness.calls} @ gen:{generation+1}")    

print("final fitness calls")
print(fitness.calls)

Gen 1
Pool 0 top 0
Pool 1 top 5
ind 0)	 fitness:0.2558 genotype:[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,

Gen 51
Pool 0 top 0
Pool 1 top 5
ind 0)	 fitness:0.3494 genotype:[0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0